# This is a work-in-progress notebook

We wish to know this:

1. How well does the model identify the correct number of senses for the target word?
2. **How well does the model identify the correct senses for the target word?**
3. **How well does the model assign the right words to a given sense of the target word?**
4. How well does the model assign the senses to the time intervals for the target word?

The script will evaluate **Q2** and **Q3**. Q4 will follow.

In [41]:
# Basic variables and imports:

import codecs, csv, os, time, re, io
from os import listdir
from os.path import isfile, join

# directories
dir_in = os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), "evaluation", "evaluation_input"))
dir_out = os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), "evaluation", "evaluation_output"))


s_senses = io.open(dir_in+"/senses_69419.txt","r")
k_senses = io.open(dir_in+"/mus.dat","r")

# DEBUG:
# s_senses = io.open(dir_in+"/senses_69419_debug.txt","r")
# k_senses = io.open(dir_in+"/mus_debug.dat","r")
# k0 = mus4
# k1 = mus3
# k2 = mus2
# k3 = mus1
# k4 = nothing

file_senses = s_senses.readlines()[1:]
output_senses = k_senses.read()

## TODO

- ~~create the notebook~~
- ~~organise the notebook~~
- ~~write "general idea" pseudocode for the evaluation~~
- ~~get input files~~
- ~~figure out data structures to store the variables~~
- write actual code

## Q2: How well does the model identify the correct senses for the target word?

### Pseudocode 

In [2]:
# For each target word, we have a list of senses  s (given by the expert)
# For each target word, we have a list of senses k (given by the model)
# This Q consists in matching s and k, and doing so in a confident way --> confidence score

for each k:
    for each s:
        create conf(k,s)

# What is conf(k,s)?
        conf(k,s) = (p1*match(w1,s)+p2*match(w1,s)+px(wx,s))/10 WHERE
    
            px = probability of word wx 
                
                and
            
            match(wx,s) =   1/number_of_senses_assigned_to_wx if s_is_one_of_them 
            
                    or 
                            0 if w_is_not_associated_to_s
                
# Once we have gone through all s for one k, we have to choose the best k for s. How? (TBD, cfr Valerio and Barbara)

# Once all ks have been assigned to all ss (or NA), we can calculate a general confidence score for the model.
# One easy way to do that: 

conf_score_model = number_of_non_NA/k

SyntaxError: invalid syntax (<ipython-input-2-82fa54601c9a>, line 5)

### Real code

Steps:

- extract all senses from the file
- use those senses as keys for a dictionary, `dict_of_words`
- fill the dictionary: for each key, we store a list of words pertaining to that sense
- transform the lists as sets so as to remove duplicates within the same sense
- create a dictionary with a word as a key and its weight as a value, depending on how many senses it appears
- parse the model output and get the probability weights for each word
- do not take into account the first line
- take care of empty lines

Todo:



In [42]:
expert_senses = list() # list where we store all sense ids provided by expert
for s in file_senses: # 60 for testing purposes
    s = s.split("\t")
    sense = s[11] # The sense ID is after the 10th tab
    expert_senses.append(sense)
    
#print(len(expert_senses),expert_senses,len(set(expert_senses)))

expert_senses = list(set(expert_senses)) # we only keep the unique senses
number_of_s = len(expert_senses)  # we create a variable that stores the number of unique senses
print("Number of senses:",number_of_s)

# This dictionary has a sense as a key, and a list of words as a value. 
dict_of_words = dict()
# This list stores all words
list_of_all_words = list()
# This dictionary stores all words as keys and their weight as value
word_weight = dict()

for i in range(0,number_of_s): # for each sense, we create a dictionary entry which has a list as value
    dict_of_words[expert_senses[i]] = list()

    for s in file_senses: # we go back in the file
        s = s.split("\t") # splitting on tabs
        
        sentence_of_ids = s[8] # 8 is for IDs, 9 is for words
        list_of_ids = sentence_of_ids.split(" ")  # splitting on spaces
        for word_id in list_of_ids:
            if s[11] == expert_senses[i]:      # we store all words for one sense 
                dict_of_words[expert_senses[i]].append(word_id)
            list_of_all_words.append(word_id) # we store all words, we'll iterate over that for scores
        

    # Here, we remove duplicates
    #dict_of_words[expert_senses[i]].append("79223") #testing
    dict_of_words[expert_senses[i]] = set(dict_of_words[expert_senses[i]]) 
      
    print(expert_senses[i],set(dict_of_words[expert_senses[i]]))
    print("\n\n")


Number of senses: 4
mus-1 {'16430', '54812', '18937', '7944', '5521', '21812', '12641', '112472', '76335', '42890', '81343', '101716', '24003', '59708', '48867', '4670', '31562', '21431', '66746', 'nlsj47984', '23690', '3986', '34855', '50093', '38547', '104872', '23468', '95853', '74127', '105344', '94483', '71262', '5009', '21802', '68970', '12349', '113060', '75910', '19641', '403', '34603', '26197', 'nlsj80462', '108536', '11072', '107167', '25007', 'nlsj68228', '37851', '62205', '63814', '65565', '2224', '66173', '55997', '678', '17730', '75948', '92662', '84725', '80557', '21487', '79595', '5112', '52332', '42842', '97285', '37776', '24801', '27629', '31948', '53956', '106502', '74735', '80042', '17381', 'nlsj9526', '7498', '116059', '104355', '28360', '80664', '40545', '13427', '6174', '13039', 'nlsj98815', 'nlsj8979', '19268', '79223', '92406', '31491', '10056', '103972', '54113', '95074', '46176', '93449', '41502', '70105', '2219', '260', '15047', '26943', '42887', '51241', 'n

In [43]:
# For every word in the list of words that we have
# we count the number of senses it appears in
# we use that number to divide its importance: 1 sense = 1 importance; 2 senses = 0.5 importance
# this can be finetuned

for word in list_of_all_words:
    x = 0
    for i in range(0,number_of_s):
        if word in dict_of_words[expert_senses[i]]:
            x += 1 
        if x != 0:
            word_weight[word] = 1/x
        
    
    print(word,word_weight[word])  

36165 1.0
18334 0.5
34855 1.0
41633 1.0
71314 1.0
69419 0.25
15380 1.0
42890 1.0
110302 1.0
104311 1.0
16400 0.5
76184 1.0
109730 1.0
110606 1.0
22997 1.0
83253 1.0
85306 0.3333333333333333
71308 0.5
19546 1.0
51241 0.5
50093 1.0
47665 1.0
63143 1.0
45996 0.25
113560 0.5
69419 0.25
62204 0.3333333333333333
22739 1.0
84534 0.5
22209 0.5
103012 1.0
23468 1.0
13579 1.0
45996 0.25
95258 1.0
19546 1.0
64914 1.0
80557 1.0
57262 0.5
29828 1.0
23468 1.0
110598 1.0
75263 1.0
69419 0.25
19641 0.5
nlsj78558 1.0
80761 1.0
23468 1.0
71308 0.5
22502 1.0
95258 1.0
26447 0.5
22100 1.0
106502 0.5
30911 1.0
69419 0.25
22100 1.0
22209 0.5
53442 1.0
8909 0.5
nlsj52509 1.0
19641 0.5
106502 0.5
75263 1.0
nlsj71743 0.5
37776 1.0
50824 1.0
nlsj78558 1.0
4335 1.0
77928 1.0
100693 1.0
22502 1.0
8665 0.5
41918 0.5
31236 0.5
64448 1.0
51661 1.0
30911 1.0
29828 1.0
62204 0.3333333333333333
75263 1.0
22209 0.5
7177 1.0
75954 1.0
82665 1.0
69419 0.25
22209 0.5
22100 1.0
53956 1.0
19641 0.5
22209 0.5
63314 1.0
31491 

34366 0.5
31562 0.3333333333333333
69419 0.25
54812 1.0
115810 1.0
12641 1.0
24340 0.5
115213 1.0
63772 0.5
34522 0.5
114753 0.5
98747 1.0
58444 1.0
24436 1.0
31236 0.5
67132 1.0
86112 1.0
31562 0.3333333333333333
69419 0.25
104421 0.5
96106 1.0
92927 1.0
79223 0.5
nlsj56179 1.0
104429 1.0
1377 0.5
11072 0.5
45996 0.25
12210 1.0
26886 1.0
112350 1.0
109757 1.0
4548 0.5
114558 1.0
71308 0.5
44552 1.0
78297 1.0
94365 1.0
2392 1.0
22209 0.5
76516 1.0
94365 1.0
28349 1.0
111764 1.0
22209 0.5
57466 1.0
116408 1.0
67009 1.0
61245 1.0
57460 1.0
69419 0.25
57460 1.0
75595 1.0
111764 1.0
69419 0.25
57466 1.0
35417 1.0
110639 0.3333333333333333
94365 1.0
75595 1.0
98869 1.0
76513 1.0
46646 0.5
42214 0.5
61176 0.5
90166 1.0
109733 1.0
39016 1.0
69419 0.25
13376 1.0
75989 1.0
100379 1.0
90504 0.5
69419 0.25
20621 0.5
19528 1.0
63772 0.5
50908 1.0
34982 1.0
57466 1.0
74735 0.5
101714 1.0
69419 0.25
112703 1.0
74752 1.0
nlsj177 1.0
74735 0.5
50908 1.0
57460 1.0
36216 1.0
97241 1.0
31607 1.0
114083 1

22209 0.5
13852 1.0
88855 1.0
37720 1.0
38472 1.0
75005 1.0
75005 1.0
63873 1.0
63873 1.0
101851 1.0
13315 1.0
13942 1.0
13813 1.0
66761 1.0
6898 1.0
48504 1.0
66750 1.0
73672 0.5
45996 0.25
69419 0.25
111764 1.0
70495 1.0
75595 1.0
nlsj71743 0.5
43975 1.0
45245 1.0
66755 1.0
26034 1.0
73672 0.5
45996 0.25
37405 1.0
48504 1.0
69419 0.25
111764 1.0
70495 1.0
75595 1.0
nlsj71743 0.5
43975 1.0
45245 1.0
74463 1.0
45996 0.25
33238 1.0
45996 0.25
41570 1.0
114387 1.0
35570 0.5
70495 1.0
14266 1.0
55497 1.0
116416 0.3333333333333333
69419 0.25
nlsj82141 1.0
18062 1.0
nlsj79694 1.0
83834 1.0
83732 1.0
2898 1.0
66494 0.3333333333333333
69419 0.25
58429 0.5
15011 1.0
70495 1.0
69419 0.25
36216 1.0
109729 1.0
20830 1.0
51785 1.0
13985 1.0
113560 0.5
101851 1.0
26039 1.0
69419 0.25
20836 1.0
55391 1.0
39180 1.0
39180 1.0
97605 1.0
621 1.0
69419 0.25
20836 1.0
101851 1.0
45996 0.25
67526 1.0
39180 1.0
58142 1.0
22660 1.0
53015 1.0
6339 1.0
29883 0.5
69419 0.25
4493 0.5
101851 1.0
51369 0.5
75595 1

100964 1.0
31562 0.3333333333333333
37961 1.0
63550 1.0
71585 1.0
106502 0.5
13715 1.0
22209 0.5
102000 0.5
69419 0.25
7362 1.0
nlsj61519 1.0
94967 1.0
22389 1.0
38744 1.0
34243 1.0
74378 0.5
64642 1.0
101982 1.0
nlsj10202 1.0
37343 1.0
76910 0.5
49875 0.5
54990 1.0
58271 1.0
60317 1.0
69419 0.25
nlsj86871 1.0
101982 1.0
66342 0.5
75653 1.0
nlsj70384 1.0
93842 1.0
53254 1.0
16440 1.0
75652 1.0
113881 1.0
nlsj70384 1.0
101982 1.0
69419 0.25
nlsj86871 1.0
60317 1.0
68144 1.0
58271 1.0
69419 0.25
75652 1.0
69419 0.25
nlsj3072 1.0
73672 0.5
45525 1.0
60316 1.0
19789 1.0
113881 1.0
62274 1.0
57262 0.5
104624 0.5
45182 1.0
17228 1.0
69419 0.25
113881 1.0
60316 1.0
30564 1.0
53254 1.0
58262 1.0
45980 0.5
90303 1.0
67364 1.0
83212 1.0
104624 0.5
69419 0.25
nlsj2729 0.5
nlsj114757 0.5
75653 1.0
110639 0.3333333333333333
83212 1.0
75653 1.0
69419 0.25
60317 1.0
101982 1.0
58262 1.0
nlsj70384 1.0
103773 1.0
19236 1.0
60317 1.0
104637 1.0
92051 1.0
103773 1.0
62258 1.0
83212 1.0
62535 1.0
75653 1.

95853 1.0
8909 0.5
18128 1.0
6325 1.0
43124 1.0
104355 0.5
65628 1.0
52571 0.3333333333333333
19641 0.5
15858 1.0
101445 1.0
70550 1.0
61791 1.0
83253 1.0
100524 1.0
nlsj114103 1.0
19891 1.0
80043 1.0
54399 1.0
55499 1.0
104355 0.5
71308 0.5
51815 1.0
93435 1.0
62204 0.3333333333333333
114553 1.0
48291 0.5
112559 0.5
69419 0.25
67507 1.0
6616 1.0
38547 1.0
72287 0.5
15378 1.0
111895 1.0
69419 0.25
76703 1.0
1564 1.0
100965 1.0
12973 1.0
58478 1.0
34372 1.0
35166 1.0
84234 0.5
72610 0.5
55981 1.0
85876 1.0
55313 1.0
19972 0.5
83869 0.5
20836 1.0
69419 0.25
104421 0.5
51373 1.0
84434 1.0
1564 1.0
98234 0.5
112070 1.0
63827 1.0
51647 1.0
76157 1.0
69419 0.25
15380 1.0
114587 1.0
104421 0.5
39847 1.0
67250 1.0
37244 1.0
83756 1.0
48704 0.5
69419 0.25
83774 1.0
49875 0.5
63827 1.0
71065 0.5
84534 0.5
3289 1.0
49933 1.0
85953 0.5
84475 1.0
8665 0.5
103221 1.0
80664 1.0
3237 1.0
nlsj1499 1.0
104421 0.5
9757 1.0
63352 1.0
25018 1.0
28355 1.0
83251 1.0
69419 0.25
91944 1.0
12641 1.0
31556 1.0
8

nlsj69856 1.0
85306 0.3333333333333333
69419 0.25
69252 1.0
38732 1.0
18937 1.0
108537 1.0
17007 1.0
48498 1.0
95819 1.0
14181 1.0
nlsj80462 1.0
24003 1.0
48291 0.5
91516 1.0
31161 1.0
nlsj32160 0.5
49589 0.5
13098 1.0
95522 1.0
80327 1.0
11305 1.0
69419 0.25
38966 1.0
91800 1.0
70958 0.5
nlsj12527 1.0
18334 0.5
nlsj80462 1.0
46804 0.5
4927 1.0
97147 1.0
7159 1.0
69419 0.25
18166 1.0
83774 1.0
45671 1.0
76184 1.0
16400 0.5
26136 1.0
104311 1.0
70482 1.0
26684 1.0
69419 0.25
68174 1.0
86429 1.0
nlsj96345 1.0
113711 1.0
22209 0.5
88498 1.0
69419 0.25
12641 1.0
38966 1.0
13608 1.0
65975 1.0
67762 0.3333333333333333
11970 1.0
19972 0.5
35570 0.5
79697 1.0
62204 0.3333333333333333
69419 0.25
72928 1.0
67868 1.0
nlsj7583 1.0
62204 0.3333333333333333
69419 0.25
65097 1.0
nlsj6617 1.0
37851 1.0
31161 1.0
110598 1.0
71308 0.5
2340 1.0
54946 1.0
69419 0.25
16609 1.0
71559 1.0
nlsj5437 1.0
69419 0.25
nlsj33192 0.5
2749 1.0
62204 0.3333333333333333
59588 1.0
nlsj106503 1.0
43977 1.0
54971 1.0
8453

72627 1.0
40545 1.0
52095 1.0
110639 0.3333333333333333
nlsj4784 1.0
83869 0.5
94474 1.0
47964 1.0
20728 1.0
nlsj4320 1.0
69419 0.25
98173 1.0
51300 1.0
23283 1.0
18271 1.0
nlsj4320 1.0
51300 1.0
98173 1.0
69419 0.25
23283 1.0
18271 1.0
31562 0.3333333333333333
79223 0.5
110639 0.3333333333333333
67762 0.3333333333333333
nlsj40132 1.0
104225 1.0
5607 1.0
41619 1.0
69419 0.25
50073 1.0
114972 1.0
6684 1.0
26684 1.0
31236 0.5
46074 1.0
76431 1.0
48670 1.0
61791 1.0
76431 1.0
103942 1.0
80761 1.0
54592 1.0
104655 0.5
nlsj5738 1.0
70708 1.0
106974 1.0
69419 0.25
27764 1.0
65983 0.5
83113 1.0
83253 1.0
12641 1.0
69419 0.25
43060 1.0
104690 0.5
34366 0.5
31562 0.3333333333333333
69419 0.25
54812 1.0
115810 1.0
12641 1.0
24340 0.5
115213 1.0
63772 0.5
34522 0.5
114753 0.5
98747 1.0
58444 1.0
24436 1.0
31236 0.5
67132 1.0
86112 1.0
31562 0.3333333333333333
69419 0.25
104421 0.5
96106 1.0
92927 1.0
79223 0.5
nlsj56179 1.0
104429 1.0
1377 0.5
11072 0.5
45996 0.25
12210 1.0
26886 1.0
112350 1.0
1

#### parsing output.dat
- split on "===============  per time  ===============" and keep first part
- transform that into a list, then
- get lines that start with "p(w|s)"
- count those, k = that number
- split the line on ":", keep the second part
- split the rest on ";", it's [ID] = prob_from_this_ID

In [44]:
lines_output = output_senses.split("===============  per time  ===============")[0].split("\n")

number_of_the_k = 0

k_words_with_prob = dict()

for line in lines_output:
    if line[:6] == "p(w|s)":
        line = line.split(":")[1]
        line = line.split(";")
        print(number_of_the_k,line)
        dico_word_prob = dict()
        k_words_with_prob[number_of_the_k] = list()
        
        line = line[:-1] # last item of the list is empty
        for word_prob in line:
        
            word_prob = word_prob.split(",")
            for word in word_prob:
                matchObj = re.findall("([\d.\w]*)",word)
                if matchObj:
                    
                    matchObj = list(filter(None,matchObj))
                    
            
            print("word id",matchObj[0],"; probability",matchObj[1])
            dico_word_prob[matchObj[0]] = matchObj[1]
        k_words_with_prob[number_of_the_k] = dico_word_prob

        number_of_the_k += 1
        



0 ['  28355 (0.088) ', ' 69419 (0.069) ', ' 57460 (0.056) ', ' 114587 (0.042) ', ' 42071 (0.041) ', ' 35267 (0.035) ', ' 51647 (0.035) ', ' 64448 (0.023) ', ' 45980 (0.018) ', ' 53826 (0.017) ', ' ']
word id 28355 ; probability 0.088
word id 69419 ; probability 0.069
word id 57460 ; probability 0.056
word id 114587 ; probability 0.042
word id 42071 ; probability 0.041
word id 35267 ; probability 0.035
word id 51647 ; probability 0.035
word id 64448 ; probability 0.023
word id 45980 ; probability 0.018
word id 53826 ; probability 0.017
1 ['  79223 (0.063) ', ' 92927 (0.057) ', ' 46574 (0.038) ', ' 67660 (0.038) ', ' 103085 (0.036) ', ' 86112 (0.030) ', ' 101982 (0.029) ', ' 75808 (0.026) ', ' 68539 (0.024) ', ' 54607 (0.024) ', ' ']
word id 79223 ; probability 0.063
word id 92927 ; probability 0.057
word id 46574 ; probability 0.038
word id 67660 ; probability 0.038
word id 103085 ; probability 0.036
word id 86112 ; probability 0.030
word id 101982 ; probability 0.029
word id 75808 ; pr

#### k_words_with_prob
This dictionary has the sense number 'k' as keys and the a dictionary of [word] = probability as values.
Example below.

In [45]:
print("Probability for word ID 5390 in sense k = 4:",k_words_with_prob[4]["5390"])

Probability for word ID 5390 in sense k = 4: 0.019


In [46]:
for key in k_words_with_prob.keys():
    print("output sense",key)
    for i in range(0,number_of_s):
        print("\texpert sense number ", i, expert_senses[i])
        for second_key in k_words_with_prob[key].keys(): # Barbara's note: shouldn't it be k_words_with_prob[i] here?
            print("\t\tword from annotation for sense", key, ":", second_key)
            if second_key in dict_of_words[expert_senses[i]]:
                print("\t\t\tword ", second_key, "is in output for sense", key, "with probability:", k_words_with_prob[key][second_key], "and weight:", word_weight[second_key])

                
# Here we get all the senses and for each sense we do a matching between the k words and s words and get the probability
# For some reason the first word for each sense arrives several times


output sense 0
	expert sense number  0 mus-1
		word from annotation for sense 0 : 28355
			word  28355 is in output for sense 0 with probability: 0.088 and weight: 1.0
		word from annotation for sense 0 : 69419
			word  69419 is in output for sense 0 with probability: 0.069 and weight: 0.25
		word from annotation for sense 0 : 57460
		word from annotation for sense 0 : 114587
			word  114587 is in output for sense 0 with probability: 0.042 and weight: 1.0
		word from annotation for sense 0 : 42071
		word from annotation for sense 0 : 35267
			word  35267 is in output for sense 0 with probability: 0.035 and weight: 1.0
		word from annotation for sense 0 : 51647
			word  51647 is in output for sense 0 with probability: 0.035 and weight: 1.0
		word from annotation for sense 0 : 64448
			word  64448 is in output for sense 0 with probability: 0.023 and weight: 1.0
		word from annotation for sense 0 : 45980
			word  45980 is in output for sense 0 with probability: 0.018 and weight: 0.5
		wor

In [47]:
## Calculating confidence score for each (words_of_k,words_of_s) pair

# conf(k,s) = (p1*match(w1,s)+p2*match(w1,s)+px(wx,s))/10
        # match(wx,s) =   1/number_of_senses_assigned_to_wx if s_is_one_of_them 

##### TODO: for now conf[k,s] is multiplied by the number of expert senses --- FIX
    
print("number of expert senses s:",number_of_s)
print("number of model output senses k:",len(k_words_with_prob.keys()))
compteur = 0

match = dict()
conf = dict()
for k in k_words_with_prob.keys():              # for each output sense, we go through...
    for s in range(0,number_of_s):       # each expert sense
        
        conf[k,s] = 0 
        
        #print("expert sense",s)
        for mot in k_words_with_prob[k]:      # for each word within output by the model for the output sense
            #print(k,mot)
            #print(expert_senses[s])
            
            if mot in dict_of_words[expert_senses[s]]:  # if that word exists in the list of expert words for that sense
                
                #print(s,dict_of_words[expert_senses[s]])
                #print(k_words_with_prob[k][mot])
                
                for word in list_of_all_words:  # this help getting a key for a dictionary later on
                    if mot == word:
                        match_weighted = float((k_words_with_prob[k][mot]))*word_weight[word] #this dictionary cfr comment on line 24
                        # word_weight[word] is already "1/number_of_expert_senses_assigned_to_this_word"
                        
                        #print("sense",expert_senses[s],"word",word,"match_weighted",match_weighted)
                        
                        #print(k,s,conf[k,s])

                        
                        # To fix? 
                        # The way the code works is that all matches happen number_of_s times
                        # (number_of_s = number of expert senses)
                        # easy fix is to divide the match score by number_of_s
                        
                        conf[k,s] = conf[k,s] + match_weighted/4
                        
                    #else: 
                        #print(word,"has no match for sense",expert_senses[s])
                        #print(word,word_weight[word],"match",k_words_with_prob[k][mot],"match weighted",match_weighted)
                    #print("test1")
                #print("test2")
                
                    #compteur += 1
                
        if (k,s) in conf.keys():
        
            conf[k,s] = conf[k,s]/10  # 10 is hardcoded, that's the number of words the model outputs
            print("k =",k,"\t s =",s,"(= expert sense",expert_senses[s],")\t conf[k,s] =",conf[k,s])
            
            #print(compteur)
            
    #print(k_words_with_prob[k])


number of expert senses s: 4
number of model output senses k: 5
k = 0 	 s = 0 (= expert sense mus-1 )	 conf[k,s] = 0.3544750000000042
k = 0 	 s = 1 (= expert sense mus-4 )	 conf[k,s] = 0.34347500000000364
k = 0 	 s = 2 (= expert sense w )	 conf[k,s] = 0.3242750000000038
k = 0 	 s = 3 (= expert sense mus-2 )	 conf[k,s] = 0.325275000000004
k = 1 	 s = 0 (= expert sense mus-1 )	 conf[k,s] = 0.012599999999999995
k = 1 	 s = 1 (= expert sense mus-4 )	 conf[k,s] = 0.0072
k = 1 	 s = 2 (= expert sense w )	 conf[k,s] = 0.021300000000000003
k = 1 	 s = 3 (= expert sense mus-2 )	 conf[k,s] = 0.022700000000000015
k = 2 	 s = 0 (= expert sense mus-1 )	 conf[k,s] = 0.01840000000000001
k = 2 	 s = 1 (= expert sense mus-4 )	 conf[k,s] = 0.019500000000000003
k = 2 	 s = 2 (= expert sense w )	 conf[k,s] = 0.0
k = 2 	 s = 3 (= expert sense mus-2 )	 conf[k,s] = 0.08210000000000009
k = 3 	 s = 0 (= expert sense mus-1 )	 conf[k,s] = 0.04556666666666648
k = 3 	 s = 1 (= expert sense mus-4 )	 conf[k,s] = 0.0

Barbara's note: "key" and "i" may be different senses; for example, for key=0, this is the first sense in the output, and i=0 is the first sense annotated by the expert. I think what we want here is to try matching all key values with all i values; for each (key, i) pair, we get the conf(key, i), as in pages 5 ff. of the Goals and plan.docx document.

Then, when possible, we can pick the best "i" for each "key"; we haven't yet decided how, but it will probably have to do with the maximum conf value.

Once we have a key-->i mapping, we can calculate precision and recall..

## Q3: How well does the model assign the right words to a given sense of the target word?

In [ ]:
# For each k, we use the words given by the expert as unquestionable truth.
# Judging the model's assignment of words to a given sense becomes a question of precision and recall.

# precision is all correct w weighted by their respective probabilities / all w weighted by their probabilities
for each k:
    for each w:
        if w in expert_list:
            w_weight = p*1
            numerator += w_weight
        w_weight = p*1
        denominator += w_weight
    precision = numerator/denominator
    
# recall is all correct w weighted by their respective probabilities / all w assigned to the sense by the expert
for each k:
    for each w:
        if w in expert_list:
            w_weight = p*1
            numerator += w_weight
    denominator = len(expert_list)
    recall = numerator/denominator
    
# f-score can be used as well

for each k:
    f_score = 2 * precision * recall / (precision+recall)